In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import metrics



In [2]:
df = pd.read_csv('Datasets/Vessel_Detail_Merged.csv')


In [3]:
df['ETA']= pd.to_datetime(df['ETA'],format="%Y-%m-%d %H:%M")
df['ATA']= pd.to_datetime(df['ATA'],format="%Y-%m-%d %H:%M")

df = df.rename(columns={'Built': 'Built_Year','Month': 'Built_Month', 'Next port of call': 'NextPort', 'Side to': 'Side', 'Owner Group': 'Owner' })
df['NextPort'] = df['NextPort'].str.split(',').str[0]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Ship_ID               882 non-null    object        
 1   Vessel_Name           882 non-null    object        
 2   Service               882 non-null    object        
 3   ETA                   882 non-null    datetime64[ns]
 4   ATA                   882 non-null    datetime64[ns]
 5   Berth                 882 non-null    object        
 6   Side                  882 non-null    object        
 7   NextPort              882 non-null    object        
 8   Ref                   882 non-null    object        
 9   Delay                 882 non-null    float64       
 10  Size                  882 non-null    object        
 11  Dwt                   882 non-null    object        
 12  GT                    882 non-null    object        
 13  Flag                

In [5]:
df['Size'] = df['Size'].str.replace(',', '').astype('int32')
df['Dwt'] = df['Dwt'].str.replace(',', '').astype('int32')
df['GT'] = df['GT'].str.replace(',', '').astype('int32')
df['Built_Year'] = df['Built_Year'].astype('int32')
df['Built_Month'] = df['Built_Month'].astype('int32')

df['SOx_Scrubber'] = df['SOx_Scrubber'].replace('Pending','0')
df['SOx_Scrubber'] = df['SOx_Scrubber'].replace('Fitted','1')

df['Eco_Engine'] = df['Eco_Engine'].replace('Eco – Electronic Engine','1')
df['Eco_Engine'] = df['Eco_Engine'].replace('Eco – Electronic Engine Modern','1')

df['Alternative_Fuel'] = df['Alternative_Fuel'].replace('LNG','1')
df['Alternative_Fuel'] = df['Alternative_Fuel'].replace('Biofuel','1')


In [6]:
df.corr()

,Delay,Size,Dwt,GT,Built_Year,Built_Month,Days_Before_ETA_Info
Delay,1.000000,0.106145,0.091501,0.096932,0.112002,-0.002159,0.147549
Size,0.106145,1.000000,0.992812,0.996367,0.734371,-0.085583,0.201510
Dwt,0.091501,0.992812,1.000000,0.994756,0.726223,-0.090019,0.209266
GT,0.096932,0.996367,0.994756,1.000000,0.727166,-0.086310,0.210235
Built_Year,0.112002,0.734371,0.726223,0.727166,1.000000,-0.060299,0.096699
Built_Month,-0.002159,-0.085583,-0.090019,-0.086310,-0.060299,1.000000,0.016116
Days_Before_ETA_Info,0.147549,0.201510,0.209266,0.210235,0.096699,0.016116,1.000000


In [7]:
#df.drop(['Dwt', 'Size', 'Ref'], axis=1,inplace=True)
df.drop(['Ref'], axis=1,inplace=True)

In [8]:
df['ETA_Year'] = df['ETA'].dt.year
df['ETA_Month'] = df['ETA'].dt.month
df['ETA_Day'] = df['ETA'].dt.day
df['ETA_Weekday'] = df['ETA'].dt.weekday
df['ETA_Hour'] = df['ETA'].dt.hour




In [9]:
#df2=df[df['ETA_Month'] == 2]



In [10]:
df['Vessel_Name'].nunique()

210

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Ship_ID               882 non-null    object        
 1   Vessel_Name           882 non-null    object        
 2   Service               882 non-null    object        
 3   ETA                   882 non-null    datetime64[ns]
 4   ATA                   882 non-null    datetime64[ns]
 5   Berth                 882 non-null    object        
 6   Side                  882 non-null    object        
 7   NextPort              882 non-null    object        
 8   Delay                 882 non-null    float64       
 9   Size                  882 non-null    int32         
 10  Dwt                   882 non-null    int32         
 11  GT                    882 non-null    int32         
 12  Flag                  882 non-null    object        
 13  Built_Year          

In [12]:
#To Preserve the cycle nature of Month, both sine and cosine are need, usefull in NN

#df['ETA_Month_sine'] = np.sin(2 * np.pi * df['ETA_Month']/11.0)


In [13]:
df_covid = pd.read_csv('Datasets/UK_Covid.csv')
df_covid['date']= pd.to_datetime(df_covid['date'],format="%d-%m-%y")

df_covid.drop(['iso_code'] ,axis=1, inplace=True)
df_covid.drop(['continent'] ,axis=1, inplace=True)
df_covid.drop(['location'] ,axis=1, inplace=True)
df_covid.drop(['Year'] ,axis=1, inplace=True)
df_covid.drop(['Quarter'] ,axis=1, inplace=True)
df_covid.drop(['Month'] ,axis=1, inplace=True)
df_covid.drop(['total_cases'] ,axis=1, inplace=True)
#df_covid.drop(['date'] ,axis=1, inplace=True)

df_covid.set_index('date',inplace=True)
df_covid=df_covid.groupby(pd.Grouper(freq="M")).sum()
df_covid.reset_index(inplace=True)

df_covid['date'] = df_covid['date'].dt.strftime('%Y-%m')
df['ETA'] = df['ETA'].dt.strftime('%Y-%m')
df_Month= pd.merge(df, df_covid, how='left', left_on='ETA',right_on='date')


In [15]:
df_Month.to_csv(r'Datasets/DF_ML_Ready.csv', index = False, header = True)

In [ ]:
df_covid = pd.read_csv('Datasets/UK_Covid.csv')
df_covid['date']= pd.to_datetime(df_covid['date'],format="%d-%m-%y")
df['ETA']= pd.to_datetime(df['ETA'],format="%y-%m")

df_covid.drop(['iso_code'] ,axis=1, inplace=True)
df_covid.drop(['continent'] ,axis=1, inplace=True)
df_covid.drop(['location'] ,axis=1, inplace=True)
df_covid.drop(['Year'] ,axis=1, inplace=True)
df_covid.drop(['Quarter'] ,axis=1, inplace=True)
df_covid.drop(['Month'] ,axis=1, inplace=True)
df_covid.drop(['total_cases'] ,axis=1, inplace=True)
#df_covid.drop(['date'] ,axis=1, inplace=True)

df_covid.set_index('date',inplace=True)
df_covid=df_covid.groupby(pd.Grouper(freq="W")).sum()
df_covid.reset_index(inplace=True)

df_covid['date'] = df_covid['date'].dt.strftime('%Y-%W')
df['ETA'] = df['ETA'].dt.strftime('%Y-%W')
df_week= pd.merge(df, df_covid, how='left', left_on='ETA',right_on='date')


In [ ]:
df_week.to_csv(r'Datasets/DF_ML_Ready_WeekCases.csv', index = False, header = True)